# Introduction to BERT
## The code for the Embeddings & Tokenization from BERT is from the LAB/Exercise 7. from the course.
### Read.me: 

Due to the computational requiremnts for BEET this jupyter notebook file is made in Google Collab. This means that the CSV file in the code is unavailable for others. However, the data (CSV-FILE) can be found i the git repository.


In [1]:
#Installing Torch and Transformer to access the BERT related dependencies from Hugging Face
!pip install torch
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 55.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 62.1 MB/s eta 0:00:00


In [2]:
#Model Related Dependencies
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler

import torch
from transformers import AutoTokenizer, AutoModel

In [40]:
#Google Related Dependencies
from pydrive.auth import GoogleAuth
from google.colab import drive
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials




In [4]:
#Code from google to access personal files on Google Drive
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

#Creating a variable for the Google File
file_id = '1Df53wMpVmFkEmp7_rBaIEyUy5ZGzrOMj' #<-- You add in here the id from you google drive file, you can find it

#Downloading the file
download = drive.CreateFile({'id': file_id})


# Download the file to a local disc
download.GetContentFile('data_preprocessed.csv')
df  = pd.read_csv("data_preprocessed.csv")
df.head()

,text,ratings.overall
0,rooms are fine service tries hard but does not...,3.0
1,best place to stay in nyc want to go back miss...,5.0
2,pleasant very pleasant clean room nice bathroo...,4.0
3,good family hotel in a good neighborhood hotel...,3.0
4,we had a beautiful deluxe room with amazing vi...,5.0


In [5]:
#Creating new classes for the hotel review ratings
df['ratings.overall'] = df['ratings.overall'].replace(range(0, 3), 'Negative')
df['ratings.overall'] = df['ratings.overall'].replace(3, 'Neutral')
df['ratings.overall'] = df['ratings.overall'].replace(range(4, 6), 'Positive')

result = df.groupby('ratings.overall').size()

In [6]:
#Checking the dataframe
df

,text,ratings.overall
0,rooms are fine service tries hard but does not...,Neutral
1,best place to stay in nyc want to go back miss...,Positive
2,pleasant very pleasant clean room nice bathroo...,Positive
3,good family hotel in a good neighborhood hotel...,Neutral
4,we had a beautiful deluxe room with amazing vi...,Positive
...,...,...
23476,close to the capitol and to the museum of art ...,Positive
23477,overall friendly feel employees want to be the...,Positive
23478,clean and quiet rooms wifi was easy to setup b...,Positive
23479,great new embassy suites service staff excelle...,Positive


In [7]:
#Condensing the size of the dataframe to ensure that BERT wont crash during the retrival of embeddings
df = df.head(8000)

# Lab 7
## This section of the Code is from LAB 7. It is adjusted to fit the project groups dataframe

In [8]:
#Defining the pre-trained model from BERT
MODEL_NAME = 'distilbert-base-uncased'



In [9]:
#Applyning the Tokenization to the model.
model = AutoModel.from_pretrained(MODEL_NAME)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [10]:
print(model)

DistilBertModel(
  (embeddings): Embeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer): Transformer(
    (layer): ModuleList(
      (0-5): 6 x TransformerBlock(
        (attention): MultiHeadSelfAttention(
          (dropout): Dropout(p=0.1, inplace=False)
          (q_lin): Linear(in_features=768, out_features=768, bias=True)
          (k_lin): Linear(in_features=768, out_features=768, bias=True)
          (v_lin): Linear(in_features=768, out_features=768, bias=True)
          (out_lin): Linear(in_features=768, out_features=768, bias=True)
        )
        (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (ffn): FFN(
          (dropout): Dropout(p=0.1, inplace=False)
          (lin1): Linear(in_features=768, out_features=3072, bias=True)
          (lin2): Li

In [11]:
print("Tokenizing...")
tokenized = tokenizer(
    df["text"].tolist(),
    add_special_tokens=True,
    padding='longest',
    return_attention_mask=True,
    return_tensors='pt',
    verbose=True
)
tokenized

Tokenizing...


{'input_ids': tensor([[ 101, 4734, 2024,  ...,    0,    0,    0],
        [ 101, 2190, 2173,  ...,    0,    0,    0],
        [ 101, 8242, 2200,  ...,    0,    0,    0],
        ...,
        [ 101, 2200, 3835,  ...,    0,    0,    0],
        [ 101, 3309, 2001,  ...,    0,    0,    0],
        [ 101, 2190, 3643,  ...,    0,    0,    0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [12]:
def get_bert_embeddings(model, tokenized):

    """ Calculate BERT embeddings for a batch of sentences.
    NOTE: Calculating BERT embeddings is a very expensive operation.
    Particularly on CPU, it can take a long time to calculate embeddings for
    a large batch of sentences (Max 10-20 minutes for 6K sentences).

    Args:
        model (transformers BERT model): BERT model.
        tokenized (dict): Dictionary of tokenized sentences (input_ids and attention_mask)

    Returns:
        n-d NumPy array: BERT embeddings for the sentences in the batch.
    """

    print("Getting model encodings...")
    # The following is a context-manager that disables gradient calculation.
    # Disabling gradient calculation is useful for inference, when you are 
    # sure that you will not call Tensor.backward(). It will reduce memory 
    # consumption for computations that would otherwise have requires_grad=True.
    # TLDR: calculating gradients is expensive. We don't need them for inference.
    with torch.no_grad():
        last_hidden_states = model(**tokenized)

    # last_hidden_states[0] is the last hidden state of the first token of the
    # sequence (classification token) further processed by a Linear layer and 
    # a Tanh activation function. The Linear layer weights are trained from the
    #  next sentence prediction (classification) objective during pretraining.
    # last_hidden_states[0].shape = (batch_size, hidden_size)
    print("Returning embeddings...")

    
    return last_hidden_states[0][:,0,:].numpy()

embeddings = get_bert_embeddings(model, tokenized)
embeddings.shape

Getting model encodings...
Returning embeddings...


(8000, 768)

# Logistic Regression

In [14]:
#Libraries
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer

In [15]:
# Split the data into training and testing sets
labels = df['ratings.overall']

#The project group chose a different Train-Test split when using BERT due to accesible amount of data.
X_train, X_test, y_train, y_test = train_test_split(embeddings,labels,train_size=0.75,stratify=labels, random_state=42) #75% train data


In [16]:
#Testing the size of the training data
print(X_train.shape)

(6000, 768)


In [17]:
# Create the pipeline for logreg
pipeline_log_reg = Pipeline([
    ('lr', LogisticRegression())
])

# Paramgrid for tuning.
param_grid_log_reg = {
'lr__C': [0.1, 1.0, 10.0],
    'lr__max_iter': [100],
    'lr__solver': ['saga'],
}

# Perform grid search to tune the model
grid_search_log_reg = GridSearchCV(pipeline_log_reg, param_grid_log_reg, cv=5)

#Fitting the gridseach to the logreg
grid_search_log_reg.fit(X_train, y_train)


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which 

GridSearchCV(cv=5, estimator=Pipeline(steps=[('lr', LogisticRegression())]),
             param_grid={'lr__C': [0.1, 1.0, 10.0], 'lr__max_iter': [100],
                         'lr__solver': ['saga']})

In [18]:
# Best model
best_model_log_reg = grid_search_log_reg.best_estimator_


In [19]:
print(grid_search_log_reg.best_params_)

{'lr__C': 0.1, 'lr__max_iter': 100, 'lr__solver': 'saga'}


In [20]:
# Evaluate the best model on the test set
y_test_pred = best_model_log_reg.predict(X_test)
print("Test Set - Classification Report:")
print(classification_report(y_test, y_test_pred))


Test Set - Classification Report:
              precision    recall  f1-score   support

    Negative       0.80      0.68      0.74       133
     Neutral       0.45      0.20      0.28       157
    Positive       0.92      0.97      0.94      1710

    accuracy                           0.89      2000
   macro avg       0.72      0.62      0.65      2000
weighted avg       0.87      0.89      0.88      2000



# Random Forrest

In [21]:
# Define pipeline
pipeline_rf = Pipeline([
    ('rf', RandomForestClassifier()),
])

In [22]:
# Define parameters for grid search
parameters_rf = {
       'rf__criterion': ['gini', 'entropy', 'log_loss'],
    'rf__n_estimators': [100, 200, 300],
    'rf__max_depth': [60,70], 
    'rf__min_samples_leaf':[5],
    'rf__class_weight':['balanced'],
}

In [23]:
# Create grid search object
grid_search_rf = GridSearchCV(pipeline_rf, parameters_rf, scoring='f1_weighted', cv=5, n_jobs=-1)

# Fit the grid search to the training data
grid_search_rf.fit(X_train, y_train)

# Print best parameters and score
print("Best parameters: ", grid_search_rf.best_params_)
print("Best accuracy score: ", grid_search_rf.best_score_)

Best parameters:  {'rf__class_weight': 'balanced', 'rf__criterion': 'gini', 'rf__max_depth': 70, 'rf__min_samples_leaf': 5, 'rf__n_estimators': 200}
Best accuracy score:  0.8562947710212304


In [24]:
# Evaluate the best model on the val data
best_model_rf = grid_search_rf.best_estimator_

print("Best model used: ", best_model_rf)


Best model used:  Pipeline(steps=[('rf',
                 RandomForestClassifier(class_weight='balanced', max_depth=70,
                                        min_samples_leaf=5,
                                        n_estimators=200))])


In [25]:
#FINAL SCORING ON TEST DATA

# Evaluate the best model on the val data

y_pred_test_rf = best_model_rf.predict(X_test)

print("Best model used: ", best_model_rf)

print(classification_report(y_test, y_pred_test_rf))

Best model used:  Pipeline(steps=[('rf',
                 RandomForestClassifier(class_weight='balanced', max_depth=70,
                                        min_samples_leaf=5,
                                        n_estimators=200))])
              precision    recall  f1-score   support

    Negative       0.74      0.53      0.61       133
     Neutral       0.62      0.05      0.09       157
    Positive       0.89      0.99      0.94      1710

    accuracy                           0.88      2000
   macro avg       0.75      0.52      0.55      2000
weighted avg       0.86      0.88      0.85      2000



# MLP

In [26]:
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler

In [27]:
# Define pipeline
pipeline_mlp = Pipeline([
    #('scaler', StandardScaler(with_mean=False)),  # Set with_mean=False for sparse matrices
    ('mlp', MLPClassifier())
])

In [28]:
# Define parameters for grid search
parameters_mlp = {
    'mlp__activation': ['relu', 'tanh'],
    'mlp__hidden_layer_sizes': [(50,), (100,), (200,), (300,)]
}

In [29]:
# Create grid search object
grid_search_mlp = GridSearchCV(pipeline_mlp, parameters_mlp, scoring='f1_weighted', cv=5, n_jobs=-1)

# Fit the grid search to the training data
grid_search_mlp.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=Pipeline(steps=[('mlp', MLPClassifier())]),
             n_jobs=-1,
             param_grid={'mlp__activation': ['relu', 'tanh'],
                         'mlp__hidden_layer_sizes': [(50,), (100,), (200,),
                                                     (300,)]},
             scoring='f1_weighted')

In [30]:
# Evaluate the best model on the val data
best_model_mlp = grid_search_mlp.best_estimator_


In [31]:
# Train accuracy
y_pred_train = best_model_mlp.predict(X_train)
train_accuracy = accuracy_score(y_train, y_pred_train)
print("Training Accuracy:", train_accuracy)

# Test accuracy
y_pred_test = best_model_mlp.predict(X_test)
test_accuracy = accuracy_score(y_test, y_pred_test)
print("Test Accuracy:", test_accuracy)

Training Accuracy: 1.0
Test Accuracy: 0.8745


In [32]:
#FINAL SCORING ON TEST DATA

y_pred_test_mlp = best_model_mlp.predict(X_test)

print("Best model used: ", best_model_mlp)


print(classification_report(y_test, y_pred_test_mlp))

Best model used:  Pipeline(steps=[('mlp', MLPClassifier(hidden_layer_sizes=(300,)))])
              precision    recall  f1-score   support

    Negative       0.65      0.66      0.65       133
     Neutral       0.34      0.31      0.32       157
    Positive       0.94      0.94      0.94      1710

    accuracy                           0.87      2000
   macro avg       0.64      0.64      0.64      2000
weighted avg       0.87      0.87      0.87      2000



# NB

In [45]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import MinMaxScaler


In [46]:
# Define pipeline
pipeline_nb = Pipeline([
        ('scaler', MinMaxScaler()),
    ('nb', MultinomialNB())
])


# Define parameters for grid search
parameters_nb = {
    'nb__alpha': [0.1, 0.5, 1.0],  # Smoothing parameter (Laplace smoothing)
    'nb__fit_prior': [True, False],  # Whether to learn class prior probabilities or not
    'nb__class_prior': [None, [0.25, 0.75], [0.5, 0.5]],  # Prior probabilities of the classes
}

# Create grid search object
grid_search_nb = GridSearchCV(pipeline_nb, parameters_nb, scoring='f1_weighted', cv=5, n_jobs=-1)

# Fit the grid search to the training data
grid_search_nb.fit(X_train, y_train)

# Print best parameters and score
print("Best parameters: ", grid_search_nb.best_params_)
print("Best accuracy score: ", grid_search_nb.best_score_)


Best parameters:  {'nb__alpha': 0.5, 'nb__class_prior': None, 'nb__fit_prior': True}
Best accuracy score:  0.8574445349618729


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
60 fits failed out of a total of 90.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
60 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/pipeline.py", line 405, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/naive_bayes.py", line 779, in fit
    self._update_class_log_prior(class_prior=class_prior)
  File "/usr/local/l

In [47]:
# Evaluate the best model on the val data
best_model_nb = grid_search_nb.best_estimator_

In [48]:
#FINAL SCORING ON TEST DATA

# Evaluate the best model on the val data

y_pred_test_nb = best_model_nb.predict(X_test)

print("Best model used: ", best_model_nb)

print(classification_report(y_test, y_pred_test_nb))


Best model used:  Pipeline(steps=[('scaler', MinMaxScaler()), ('nb', MultinomialNB(alpha=0.5))])
              precision    recall  f1-score   support

    Negative       0.55      0.58      0.57       133
     Neutral       0.38      0.23      0.29       157
    Positive       0.92      0.95      0.93      1710

    accuracy                           0.87      2000
   macro avg       0.62      0.59      0.59      2000
weighted avg       0.85      0.87      0.86      2000

